This is the setup cell

In [1]:
import datetime
import pathlib
import time
import math

import obspy
from obspy.signal import PPSD
import scipy
import matplotlib.pyplot as plt

#Run this line if you want interactive plots
%matplotlib qt

import numpy as np

import sprit

In [2]:
dPath = r'\\isgs-sinkhole.ad.uillinois.edu\geophysics\HVSR\LakeCalumet\LC_RaspShake'
#dPath = r'\\isgs-sinkhole.ad.uillinois.edu\geophysics\HVSR\LakeCalumet\LC_RaspShake\Test12_AM.RAC84.00.2023-02-15_2132-2200.mseed'
trimDir = r'\\isgs-sinkhole.ad.uillinois.edu\geophysics\HVSR\LakeCalumet\LC_RaspShake'
#dPath = r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\HVSR\ChampaignCo\Mahomet2023\Data\RawData"
dPath = r"C:\Users\riley\OneDrive - University of Illinois - Urbana\Data_OneDrive\Seismic\HVSR\Data\RAC84"
dPath = r"C:\Users\riley\OneDrive - University of Illinois - Urbana\Data_OneDrive\Seismic\HVSR\Data\UOFI1_AM.RAC84.00.2023-02-15_1704-1734.mseed"

params = sprit.input_param( acq_date='2023-02-15',
                            #starttime = obspy.UTCDateTime('2023-02-06T21:32:00.00'),
                            #endtime = obspy.UTCDateTime('2023-02-06T22:00:00.00'),
                            tzone = 'UTC',#or 'local', #or 'UTC'
                            dst=False,
                            lon = -88.2290526,
                            lat =  40.1012122,
                            elevation = 755,
                            site = 'HVSR Site',
                            dataPath = dPath
                            )

No metadata file specified!
Using default metadata file for Raspberry Shake v.7 contained in repository at
 c:/Users/riley/LocalData/Github/SPRIT/resources/raspshake_metadata.inv


In [9]:
params = sprit.get_metadata(params)
stream = sprit.fetch_data(params=params)

Day of Year: 46


c:\Users\riley\anaconda3\envs\seismic39\lib\site-packages\obspy\core\inventory\network.py:251: UserWarning: Found more than one matching response. Returning first.
  warnings.warn(msg)


In [4]:
xwindows = sprit.select_windows(stream)

In [5]:
stream = sprit.remove_noise(input=stream, kind='manual', window_list=xwindows)


In [10]:
params = sprit.generate_ppsds(params=params, stream=stream, ppsd_length=60, overlap=0.5, period_step_octaves=0.01)
hvsr_dict = sprit.process_hvsr(params=params, method=4, resample=False, smooth=10)
hvsr_dict = sprit.check_peaks(hvsr_dict=hvsr_dict)

In [11]:
sprit.hvplot(hvsr_dict, kind='HVSR p tp ann', show=True)